In [1]:
import os
import time
import pickle

from collections import defaultdict

import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as ly
sess_opt = tf.ConfigProto(gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.95 , allow_growth=True)
                         ,device_count={'GPU': 1})

from matplotlib import pyplot as plt

from utils import exist_or_mkdir , data_manager , transform_orig

exp_folder = "Attn_ver3"
model_path = "model_para"
tmp_path = "tmp"
log_path = "log"

In [2]:
exp_folder = exist_or_mkdir("./",exp_folder)
model_path = exist_or_mkdir(exp_folder,model_path)
tmp_path = exist_or_mkdir(exp_folder,tmp_path)
log_path = exist_or_mkdir(exp_folder,log_path)

Path : './Attn_ver3'
Path : './Attn_ver3/model_para'
Path : './Attn_ver3/tmp'
Path : './Attn_ver3/log'


## Loading data

In [3]:
Encoder_max_len = 60
Decoder_max_len = 30
min_count = 3

In [4]:
train_path = ["data/{}/train.csv".format(x) for x in ["all","rhyme","length","pos"]]
test_path = ["data/{}/test.csv".format(x) for x in ["all","rhyme","length","pos"]]

In [5]:
print("### Loading Train Data ###")
data_agent = data_manager(train_path , train=True)

### Loading Train Data ###
Data count : 651339

### Data view ###
Original data  : ['SOS', '是', '你', '让', '我', '的', '心痛', 'EOS', 'm', 'p', 'm', 'a', 'NOP', 'en', 'NOE', '4', 'NOR']
Output Sentence : ['SOS', '一天', '比', '一天', '深', 'EOS']
Data count : 1302678

### Data view ###
Original data  : ['SOS', '我', '皱纹', '在', '你', '的', '笑脸', 'EOS', 'a', 'v', 'n', 'nr', 'r', 'v', 'r', 'NOP', 'e', 'NOE', '7', 'NOR']
Output Sentence : ['SOS', '最好', '没有', '人', '明白', '我', '说', '什么', 'EOS']
Data count : 1954017

### Data view ###
Original data  : ['SOS', '满身', '伤痕累累', '也', '来不及', '痛', 'EOS', 'ou', 'NOE']
Output Sentence : ['SOS', '那', '是', '指引', '我', '走向', '你', '的', '清楚', '感受', 'EOS']
Data count : 2605356

### Data view ###
Original data  : ['SOS', '口头上', '是', '男女朋友', 'EOS', 'v', 'r', 'v', 'r', 's', 'NOP']
Output Sentence : ['SOS', '送', '你', '到', '你家', '巷口', 'EOS']


In [6]:
print("### Loading Test Data ###")
test_agent = data_manager(test_path , train=False)

### Loading Test Data ###
Data count : 70000

### Data view ###
Original data  : ['SOS', '我', '好乱', '我', '好', '苦', 'EOS', 'r', 'v', 'v', 'r', 'r', 'v', 'v', 'r', 'NOP', 'o', 'NOE', '8', 'NOR']
Data count : 140000

### Data view ###
Original data  : ['SOS', '我', '多', '想', '看看', '兰兰', '的', '天', 'EOS', 'ing', 'NOE']
Data count : 210000

### Data view ###
Original data  : ['SOS', '只', '剩下', '不知疲倦', '的', '肩膀', 'EOS', '5', 'NOR']
Data count : 280000

### Data view ###
Original data  : ['SOS', '不失为', '天大', '的', '幸福', '将', '这', '一份', '礼物', 'EOS', '6', 'NOR']


## Preprocessing and Padding

In [7]:
idx_in_sen , idx_out_sen , mask_in , mask_out , length_in , idx2word , word2idx , remain_idx = \
    transform_orig([data_agent.orig_data,data_agent.out_sen],min_count=min_count,
                   max_len = [Encoder_max_len,Decoder_max_len])

Min Count : 3
Max Length : [60, 30]
Word Count : 92272
Orig data  : ['SOS', '不再', '想', '你', 'EOS', '5', 'NOR']
Index data : [1177, 119, 5, 3, 79, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Output Orig data  : ['SOS', '我', '的', '勇气', '已', '不言而喻', 'EOS']
Output Index data : [2, 36, 10, 77, 354, 56116, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
idx_in_sen.shape , idx_out_sen.shape , mask_in.shape , mask_out.shape , length_in.shape

((2605000, 60), (2605000, 30), (2605000, 60), (2605000, 30), (2605000,))

In [9]:
pickle.dump({"orig_word":[idx2word,word2idx] },
            open(os.path.join(tmp_path,"tokenizer.pkl") , "wb"))

## Build Model

In [10]:
def Encoder(inputs , dim , name , init_state=None , t_len=20 , reuse=False , stack_flag=False):
    cell = tf.contrib.rnn.LSTMCell(dim,name=name,reuse=reuse)
    if init_state:
        state = init_state
    else:
        state = [tf.zeros([tf.shape(inputs)[0] , cell.state_size[0]]),
                 tf.zeros([tf.shape(inputs)[0] , cell.state_size[1]])]
    output_seq = []
    for t in range(t_len):
        if stack_flag:
            out , state = cell(inputs[:,t] , state)
        else:
            out , state = cell(inputs[t] , state)
        output_seq.append(out)
    
    return output_seq , state

In [11]:
def attend_vector(inputs , state , mask , name):
    with tf.name_scope("Attention"):
        state = tf.tile(tf.expand_dims(state , axis=1) , [1,tf.shape(inputs)[1],1])
        concat_vec = tf.concat([inputs,state],axis=-1)
        fc1 = ly.fully_connected(concat_vec,192,activation_fn=tf.nn.leaky_relu,biases_initializer=None,
                                 scope="Attn_{}_1".format(name),reuse=tf.AUTO_REUSE)
        fc2 = ly.fully_connected(fc1,96,activation_fn=tf.nn.leaky_relu,biases_initializer=None,
                                 scope="Attn_{}_2".format(name),reuse=tf.AUTO_REUSE)
        fc3 = ly.fully_connected(fc1,1,activation_fn=None,biases_initializer=None,
                                 scope="Attn_{}_3".format(name),reuse=tf.AUTO_REUSE)
        score = tf.nn.softmax(fc3 , axis=1)
        ## define my softmax
#         exp_fc3 = tf.exp(fc3)*mask
#         exp_sum = tf.reduce_sum(exp_fc3,axis=1,keepdims=True)
#         score = exp_fc3/exp_sum
    
    return score , tf.reduce_sum(inputs*score , axis=1)

def attn_Encoder(inputs , mask , dim , name , init_state=None , t_len=20 , reuse=False):
    cell = tf.contrib.rnn.LSTMCell(dim,name=name,reuse=reuse)
    if init_state:
        state = init_state
    else:
        state = [tf.zeros([tf.shape(inputs)[0] , cell.state_size[0]]),
                 tf.zeros([tf.shape(inputs)[0] , cell.state_size[1]])]
    output_seq = []
    score_seq = []
    for t in range(t_len):
        score , attn_vec = attend_vector(inputs,state[1],mask,name="Encode")
        out , state = cell(attn_vec,state)
        output_seq.append(out)
        score_seq.append(score)
    
    return output_seq , state , score_seq 


def attn_Decoder(inputs , inputs_E , mask , dim , name , init_state=None , t_len=20 , reuse=False , stack_flag=False):
    cell = tf.contrib.rnn.LSTMCell(dim,name=name,reuse=reuse)
    if init_state:
        state = init_state
    else:
        state = [tf.zeros([tf.shape(inputs)[0] , cell.state_size[0]]),
                 tf.zeros([tf.shape(inputs)[0] , cell.state_size[1]])]
    output_seq = []
    score_seq = []
    for t in range(t_len):
        score , attn_vec = attend_vector(inputs_E,state[1],mask,name="Decode")
        if stack_flag:
            attn_vec = tf.concat([attn_vec,inputs[:,t]] , axis=-1)
        else:
            attn_vec = tf.concat([attn_vec,inputs[t]] , axis=-1)
        out , state = cell(attn_vec,state)
        output_seq.append(out)
        score_seq.append(score)
    
    return output_seq , state , score_seq 


In [12]:
def word_clf(inputs,dim,embd):
    fc1 = ly.fully_connected(inputs,dim,activation_fn=tf.nn.leaky_relu,scope="clf_fc1",reuse=tf.AUTO_REUSE)
    fc2 = ly.fully_connected(fc1,int(embd.shape[0]),activation_fn=None,scope="clf_fc2",reuse=tf.AUTO_REUSE)
    return fc2@embd

In [13]:
def mask_catece(x):
    logit = x[0]
    idx = x[1]
    ce = []
    for t in range(Decoder_max_len-1):
        ce.append( tf.log(tf.nn.embedding_lookup(logit[t],idx[t])+1e-10) )
    return tf.stack(ce)

In [14]:
Seq_g = tf.Graph()
embd_dim = 256
L0_dim = 256
L1_dim = 384
L2_dim = 384
clf_dim = 300

with Seq_g.as_default():
    with tf.name_scope("Input"):
        _in = tf.placeholder(tf.int32,[None,None])
        _in_mask = tf.placeholder(tf.float32,[None,None])
        in_mask = tf.expand_dims(_in_mask,axis=-1)
        
        _in_length = tf.placeholder(tf.int32,[None])
        
        _out = tf.placeholder(tf.int32,[None,Decoder_max_len])
        _out_mask = tf.placeholder(tf.float32,[None,Decoder_max_len])
        gt = _out[:,1::]
        gt_mask = _out_mask[:,1::]
        
        schedual_rate = tf.random_uniform([Decoder_max_len],maxval=1.0)
        schedual_th = tf.placeholder(tf.float32)
        infer_start = tf.ones([tf.shape(_in)[0]],dtype=tf.int32)
        
    with tf.name_scope("Embedding"):
        ## word embedding
        _embd = tf.Variable(tf.truncated_normal([len(idx2word) , embd_dim],stddev=0.1),name="Word_Embd")
        _embd_T = tf.transpose(_embd,[1,0])
        x_vector = tf.nn.embedding_lookup(_embd,_in,max_norm=5)
        y_vector = tf.nn.embedding_lookup(_embd,_out,max_norm=5)
        
    
    
    with tf.name_scope("Encoder"):
        e_cell0 = tf.contrib.rnn.LSTMCell(L0_dim,name="E_layer_0",reuse=False)
        e_cell1 = tf.contrib.rnn.LSTMCell(L1_dim,name="E_layer_1",reuse=False)
        
        E_layer_0 , E_state_0= tf.nn.dynamic_rnn(e_cell0,x_vector,sequence_length=_in_length,dtype=tf.float32)
        E_layer_1 , E_state_1= tf.nn.dynamic_rnn(e_cell1,E_layer_0,sequence_length=_in_length,dtype=tf.float32)
        
    with tf.name_scope("Decoder"):
        
        D_layer_0 , D_state_0 = Encoder(y_vector,L0_dim,"rnn/E_layer_0",init_state=E_state_0,reuse=True,
                                        t_len=Decoder_max_len-1,stack_flag=True)
        D_layer_1 , D_state_1 = Encoder(D_layer_0,L1_dim,"rnn/E_layer_1",init_state=E_state_1,reuse=True,
                                        t_len=Decoder_max_len-1,stack_flag=False)
        
        D_layer_1 , D_state_1 , D_score = attn_Decoder(D_layer_1,E_layer_1,in_mask,L2_dim,name="Attn_D_layer_1",
                                                       init_state=E_state_1,t_len=Decoder_max_len-1,stack_flag=False)
        
        output_seq = []
        for t in range(Decoder_max_len-1):
            choice_input = D_layer_1[t]
            out = word_clf(choice_input,clf_dim,_embd_T)
            output_seq.append(out)
        _logits = tf.stack(output_seq,axis=1)
        _prob = tf.nn.softmax(_logits,axis=-1)
        
        
    with tf.name_scope("Loss"):
#         cross_entropy_0 = tf.map_fn(mask_catece,elems=(_prob,gt),dtype=tf.float32)
#         cross_entropy = tf.reduce_sum(cross_entropy_0*gt_mask,axis=-1)/tf.reduce_sum(gt_mask,axis=-1)
#         _loss = -tf.reduce_mean(cross_entropy)

        gt = tf.one_hot(gt,depth=len(idx2word),dtype=tf.float32)
        cross_entropy_0 = tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf.reshape(gt,[-1,len(idx2word)]),
                                                                     logits=tf.reshape(_logits,[-1,len(idx2word)]))
        cross_entropy_1 = tf.reshape(cross_entropy_0,[-1,Decoder_max_len-1])
        cross_entropy = tf.reduce_sum(cross_entropy_1*gt_mask,axis=-1)/tf.reduce_sum(gt_mask,axis=-1)
        _loss = tf.reduce_mean(cross_entropy)
        
    with tf.name_scope("Train_strategy"):
        opt = tf.train.AdamOptimizer(1e-4)
        _update = opt.minimize(_loss)
    
    with tf.name_scope("Inference"):
        ## start at Encoder layer 2 : E_layer2
        infer_out = tf.nn.embedding_lookup(_embd,infer_start)
        infer_state_0 = E_state_0
        infer_state_1 = E_state_1
        infer_state_2 = E_state_1
        
        infer_score_seq = []
        infer_pred_idx_seq = []
        infer_logits_seq = []
        for t in range(Decoder_max_len-1):
            tmp = Encoder([infer_out],L0_dim,"rnn/E_layer_0",init_state=infer_state_0,reuse=True,
                          t_len=1,stack_flag=False)
            infer_layer_0 , infer_state_0 = tmp
            
            
            tmp = Encoder(infer_layer_0,L1_dim,"rnn/E_layer_1",init_state=infer_state_1,reuse=True,
                          t_len=1,stack_flag=False)
            infer_layer_1 , infer_state_1 = tmp
            
            tmp = attn_Decoder(infer_layer_1,E_layer_1,in_mask,L2_dim,name="Attn_D_layer_1",
                               init_state=infer_state_2,t_len=1,reuse=True,stack_flag=False)
            
            infer_layer_2 , infer_state_2 , infer_score = tmp
            
            infer_score_seq.append(infer_score)
            
            infer_out = word_clf(infer_layer_1[0],clf_dim,_embd_T)
            infer_logits_seq.append(infer_out)
            
            out_index = tf.argmax(infer_out,axis=1)
            infer_pred_idx_seq.append(out_index)
            infer_out = tf.nn.embedding_lookup(_embd , out_index)
            
        infer_pred_idx_seq = tf.stack(infer_pred_idx_seq,axis=1)
        infer_logits = tf.stack(infer_logits_seq,axis=1)
        infer_prob = tf.nn.softmax(infer_logits,axis=-1)
    
    tf.summary.FileWriter(log_path,graph=Seq_g)
    _init = tf.global_variables_initializer()
    saver = tf.train.Saver(max_to_keep=10,var_list=tf.global_variables())
    
print("Finish Building!!\n")

Finish Building!!



## Training

In [15]:
print("### Start Training ###\n")

### Start Training ###



In [16]:
sess = tf.Session(graph=Seq_g,config=sess_opt)
sess.run(_init)

In [17]:
def get_batch(i):
    tmp_end = max(length_in[i])
    my_dict = {
        _in:idx_in_sen[i,:tmp_end],
        _in_mask:mask_in[i,:tmp_end],
        _out:idx_out_sen[i],
        _out_mask:mask_out[i],
        _in_length:length_in[i]
    }
    return my_dict

In [18]:
def evaluate_batch(sess,_pred,count=3):
    idx = np.random.choice(idx_in_sen.shape[0],[count])
    tmp_max_len = max(length_in[idx])
    my_dict = {
        _in:idx_in_sen[idx,:tmp_max_len],
        _in_mask:mask_in[idx,:tmp_max_len],
        _in_length:length_in[idx]
    }
    pred = sess.run(_pred , feed_dict=my_dict)
    
    word_seq = []
    for i in range(3):
        idx_sen = pred[i]
        tmp = []
        for t in range(Decoder_max_len-1):
            if(idx_sen[t] == 3):
                break
            tmp.append(idx2word[idx_sen[t]])
        word_seq.append(tmp)
    
    print("Max length :" , tmp_max_len)
    for i in range(3):
        print("  Input word  :" , data_agent.orig_data[remain_idx[idx[i]]])
        print("  Input index :" , idx_in_sen[idx[i],:tmp_max_len])
        print("  Ground word :" , data_agent.out_sen[remain_idx[idx[i]]])
        print("    Output    :" , word_seq[i])
        print()

In [19]:
batch_size = 150
n_epoch = 50
n_step = idx_in_sen.shape[0]//batch_size

r_index = np.arange(idx_in_sen.shape[0])
loss_list = []
try:
    for e in range(1,n_epoch+1):
        np.random.shuffle(r_index)
        start_time = time.time()
        start = 0
        for s in range(n_step):
            idx = r_index[start:start+batch_size]
            _,l = sess.run([_update,_loss] , feed_dict=get_batch(idx))
            start += batch_size
            print("step {:>5d} loss : {:>9.4f} time : {:>7.2f}".format(s,l,time.time()-start_time) , end="\r")
            if s % 500 == 0:
                print("step {:>5d} loss : {:>9.4f} time : {:>7.2f}".format(s,l,time.time()-start_time) , end="\n")
                evaluate_batch(sess,infer_pred_idx_seq,3)

        loss_list.append(l)
        print("\nEpoch {0:>3d}/{1:d} loss : {2:>9.4f} time : {3:>8.2f}".format(e,n_epoch,l,time.time()-start_time))

        evaluate_batch(sess,infer_pred_idx_seq,3)

        if e%4 == 0:
            saver.save(sess,os.path.join(model_path,"model_{}.ckpt".format(e)))
except KeyboardInterrupt :
    saver.save(sess,os.path.join(model_path,"model_{}.ckpt".format("lastest")))
    pickle.dump(loss_list,open(os.path.join(log_path,"loss.pkl") , "wb"))
    print()
    print("Save loss history...")


step     0 loss :   11.4314 time :   11.08
Max length : 20
  Input word  : ['SOS', '心', '可以', '攻', '可以', '守', '可以', '呃', '可以', '抢', 'EOS', 'c', 'v', 'c', 'v', 'c', 'v', 'NOP']
  Input index : [  613    70   968    70   692    70 28898    70  2425     3    64    15
    64    15    64    15    18     0     0     0]
  Ground word : ['SOS', '可以', '偷', '可以', '伤', '可以', '收', 'EOS']
    Output    : ['作梦', '辞赋', '衡量', '衡量', '欠债', '清蒸鱼', '争发', '拜托', '争发', '雪上加霜', '雪上加霜', '颤抖抖', '颤抖抖', '傲漫', '傲漫', '夢境', '脑部', '脑部', '脑部', '迂回', '谈谈心', '捂上', '畸型', '畸型', '畸型', '抿', '抿', '抿', '抿']

  Input word  : ['SOS', '轰然', '的', '巨响', 'EOS', '5', 'NOR']
  Input index : [45936    10 28650     3    79    22     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
  Ground word : ['SOS', '堵住', '了', '所有', '的', '路', 'EOS']
    Output    : ['咳约', '还合', '咳约', '咳约', '还合', '还合', '大龄青年', '大龄青年', '阮入', '阮入', '洗点', '雙親', '雙親', '雙親', '雙親', '不醉不归', '不醉不归', '沧沧', '天堂', '沧沧', '先于', '先于', '范', '电影海报'

step  3000 loss :    6.0470 time : 1264.83
Max length : 20
  Input word  : ['SOS', '偶而', '刷油漆', '常', '开车', '出去', 'EOS', 'v', 'n', 'n', 'ns', 'NOP', 'i', 'NOE', '4', 'NOR']
  Input index : [ 5633 72298   781  7173  7055     3    15    31    31   135    18    32
    20    73    22     0     0     0     0     0]
  Ground word : ['SOS', '没有', '电视', '网络', '不太熟悉', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我']

  Input word  : ['SOS', '不言不语', '冷冷静静', '月光', '中度', '连', '你', '自己', 'EOS', 'r', 'n', 'v', 'p', 'r', 'v', 'a', 'NOP', 'a', 'NOE', '7', 'NOR']
  Input index : [12196 58631  3743 58632  1017     5   268     3    12    31    15    13
    12    15    40    18    40    20    66    22]
  Ground word : ['SOS', '你', '制沙', '制胜', '把', '自己', '变', '伟大', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我

step  6000 loss :    5.6481 time : 2518.78
Max length : 21
  Input word  : ['SOS', '走遍', '世界地图', '每', '寸', '方土', 'EOS', '9', 'NOR']
  Input index : [10959 57566   213 12027 57567     3    21    22     0     0     0     0
     0     0     0     0     0     0     0     0     0]
  Ground word : ['SOS', '这', '世界', '只有', '你', '只有', '我', '两', '人', '还好', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '可是', '可是', '可是']

  Input word  : ['SOS', '你', '要', '自由', '也', '要', '永远', 'EOS', 'ong', 'NOE']
  Input index : [   5  378 1010   68  378  217    3  444   20    0    0    0    0    0
    0    0    0    0    0    0    0]
  Ground word : ['SOS', '却', '忘', '了', '男人', '的', '心', '也', '会', '痛', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '可是', '可是', '可是', '可是', '可是', '可是', '然而', '然而', '然而', '然而', '然而', '然而']

  Input word  : ['SOS', 

step  9000 loss :    5.3286 time : 3763.82
Max length : 14
  Input word  : ['SOS', '养育', '的', '恩情', '比地', '大天', '高', 'EOS', '5', 'NOR']
  Input index : [ 8634    10 24307 41347 44943  1775     3    79    22     0     0     0
     0     0]
  Ground word : ['SOS', '再', '看', '如今', '的', '你', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '你', '你', '你', '你', '你', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果']

  Input word  : ['SOS', '第三幕', '穿', '泳裤', '的', '爱神', '上', 'EOS', 'd', 'v', 'd', 'v', 'd', 'v', 'NOP']
  Input index : [75372  2393 75373    10 20634   554     3    17    15    17    15    17
    15    18]
  Ground word : ['SOS', '不许', '睡觉', '尤其', '是', '不许', '睡着', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '因为', '因为', '因为', '因为', '因为', '因为', '因为']

  Input word  : ['SOS', '仍旧', '注视', '你', '眼睛', 'EOS', 'v', 'd', 'r', 'n', 'v', 'NOP']
  I

step 12000 loss :    5.0265 time : 5019.24
Max length : 21
  Input word  : ['SOS', '星星', '飞过', '要是', '要是', '你', '那', '颗', 'EOS', 'v', 'm', 'r', 'n', 'nr', 'ug', 'NOP']
  Input index : [  951  2458 13422 13422     5   432  2951     3    15    14    12    31
    39   457    18     0     0     0     0     0     0]
  Ground word : ['SOS', '可否', '许', '我', '低声', '祝你幸福', '过', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '这', '这', '这', '这', '这', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '如果', '如果', '如果']

  Input word  : ['SOS', '今天', '以后', '留下', '回忆', '我们', '可以', '温习', 'EOS', 'l', 'd', 'nr', 'r', 'm', 'q', 'p', 'v', 'NOP', 'i', 'NOE', '8', 'NOR']
  Input index : [ 1378  1503  1274   118   204    70 22728     3    63    17    39    12
    14    56    13    15    18    32    20    42    22]
  Ground word : ['SOS', '我爱你', '不', '言语', '这', '一刻', '天', '在', '哭泣', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我

...

step  3000 loss :    4.6389 time : 1169.61
Max length : 15
  Input word  : ['SOS', '再', '没', '余地', '继续', '缠绕', 'EOS', 'v', 't', 'v', 'NOP']
  Input index : [  437     7 18155  1419  8267     3    15    92    15    18     0     0
     0     0     0]
  Ground word : ['SOS', '谈情', '一世', '发现', 'EOS']
    Output    : ['别', '别', '别', '我', '我', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '每当', '每当', '每当', '每当', '每当', '每当', '当', '当', '当', '当', '当', '当', '当', '当']

  Input word  : ['SOS', '像', '每事', '完全', '明白', '我', 'EOS', 'r', 'n', 'zg', 'v', 'NOP', 'i', 'NOE', '4', 'NOR']
  Input index : [  182 24662  1596   668    36     3    12    31   212    15    18    32
    20    73    22]
  Ground word : ['SOS', '为何', '甜言', '仍', '响起', 'EOS']
    Output    : ['我', '我', '每当', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '可是', '可是', '可是', '可是', '可是', '可是', '可是', '当', '当', '当', '当']

  Input word  : ['SOS', '二十种', '以上', '的', '汉字', '字体', 'EOS', '4', 'NOR']
  Input index :

step  6000 loss :    4.5703 time : 2358.86
Max length : 28
  Input word  : ['SOS', '等等', '等', 'EOS', '2', 'NOR']
  Input index : [4416  203    3  127   22    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0]
  Ground word : ['SOS', '等等', '等', 'EOS']
    Output    : ['别', '别', '别', '别', '然后', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果']

  Input word  : ['SOS', '那么', '多年', '是', '什么', '让', '生活', '变得', '像', '易碎', '的', '泡沫', 'EOS', 'r', 'm', 'd', 'd', 'v', 'r', 'z', 'ul', 'vn', 'uj', 'n', 'NOP', 'uo', 'NOE', '11', 'NOR']
  Input index : [ 577 2939   96  139   84  609  464  182 7524   10 4259    3   12   14
   17   17   15   12  485   50   93   41   31   18  466   20  252   22]
  Ground word : ['SOS', '那么', '多年', '又', '究竟', '是', '什么', '错综', '了', '生命', '的', '脉络', 'EOS']
    Output    : ['每', '每', '每', '每', '每', '每', '每', '每', '每', 

step  9000 loss :    4.6301 time : 3649.06
Max length : 20
  Input word  : ['SOS', '放开', '你', '的', '手', '谁', '都', '不是', '谁', '的', '玩具', 'EOS', 'n', 'd', 'd', 'v', 'r', 'c', 'r', 'v', 'NOP']
  Input index : [6820    5   10 1472  258  191  543  258   10 2843    3   31   17   17
   15   12   64   12   15   18]
  Ground word : ['SOS', '头', '也', '不', '回', '我', '不管', '别人', '信不信', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我']

  Input word  : ['SOS', '又', '要', '清楚', '黯然', '度过', '一年', 'EOS', 'a', 'v', 'a', 'v', 'nr', 'r', 'r', 'd', 'v', 'n', 'v', 'v', 'NOP']
  Input index : [  253   378  1363 22638 10487   439     3    40    15    40    15    39
    12    12    17    15    31    15    15    18]
  Ground word : ['SOS', '好', '改变', '坏', '改变', '祝福', '你', '我', '都', '有', '能力', '去', '应变', 'EOS']
    Output    : ['我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我'

step 12000 loss :    4.6386 time : 4885.78
Max length : 19
  Input word  : ['SOS', '早已', '在', '酝酿', '我', '选择', '它', 'EOS', 'r', 'v', 'r', 'p', 'vn', 'v', 'NOP', 'ang', 'NOE', '6', 'NOR']
  Input index : [1180   24 8214   36  189 1285    3   12   15   12   13   93   15   18
  146   20   33   22    0]
  Ground word : ['SOS', '它', '选择', '我', '把', '生命', '传唱', 'EOS']
    Output    : ['我', '我', '这', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '我', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '经过', '经过', '经过', '经过']

  Input word  : ['SOS', '而', '我们', '围绕', '着', '在', '寻找', '中', 'EOS', 'uo', 'NOE']
  Input index : [ 886  204 9941   44   24 2982  688    3  466   20    0    0    0    0
    0    0    0    0    0]
  Ground word : ['SOS', '爱', '在', '某个', '角落', 'EOS']
    Output    : ['如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '别说', '别说', '别说', '别说', '别说', '别说']

  Input word  : ['SOS', '你'

step 15500 loss :    4.2946 time : 6340.08
Max length : 12
  Input word  : ['SOS', '我', '是', '被', '你', '囚禁', '的', '鸟', 'EOS', 'ao', 'NOE']
  Input index : [  36   96   46    5   11   10 1232    3 1235   20    0    0]
  Ground word : ['SOS', '得到', '的', '爱', '越来越少', 'EOS']
    Output    : ['然后', '如果', '如果', '手牵手', '今后', '今后', '今后', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '如果', '手牵手', '明天', '喔', '哦', '手牵手', '明天', '喔', '哦']

  Input word  : ['SOS', '有', '我们', '牵手', '的', '过去', 'EOS', 'v', 'uz', 'n', 'uj', 'n', 'NOP']
  Input index : [ 322  204 4074   10 3507    3   15   38   31   41   31   18]
  Ground word : ['SOS', '漂浮', '着', '尘埃', '的', '空气', 'EOS']
    Output    : ['不要', '把', '把', '把', '把', '我', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '你', '每当', '每当', '每当', '每当', '除', '除', '除']

  Input word  : ['SOS', '爱', '疯', '了', '感情', '一定', '会', '被', '践踏', 'EOS', 'a', 'NOE']
  Input index : [   54  1182    52   929  2350   201 

In [ ]:
print("\n### Training Finished!! ###\n")

## Inference

In [39]:
def infer_test_data(sess,_pred,x,word2idx,batch_size=1000):
    start = 0
    pred_word_seq = []
    while(start < len(x)):
        batch_idx_sen = []
        batch_length = []
        batch_mask = []
        max_len = 0
        for s in x[start : start+batch_size]:
            l = len(s)-1
            if(l>max_len):
                max_len = l
        
        for s in x[start : start+batch_size]:
            arr = []
            ## s[1::] : remove first word "SOS"
            batch_mask.append( np.zeros([max_len]))
            batch_mask[-1][0:len(s[1::])] += 1 
            batch_length.append(len(s[1::]))
            for ss in s[1::]:
                try:
                    arr.append(word2idx[ss])
                except:
                    arr.append(1)
            arr.extend([0]*(max_len-len(s[1::])))
            batch_idx_sen.append(arr)
        batch_idx_sen = np.array(batch_idx_sen)
        batch_length = np.array(batch_length)
        batch_mask = np.stack(batch_mask)
        
        pred_sen = sess.run(_pred,feed_dict={
            _in:batch_idx_sen,
            _in_length:batch_length,
            _in_mask:batch_mask
        })
        
        for i in range(batch_size):
            idx_sen = pred_sen[i]
            tmp = []
            for t in range(Decoder_max_len-1):
                if(idx_sen[t] == 3):
                    break
                elif(idx_sen[t] == 1):
                    tmp.append(np.random.choice(idx2word))
                else:
                    tmp.append(idx2word[idx_sen[t]])
            pred_word_seq.append(" ".join(tmp))
        start += batch_size
        
    return pred_word_seq

In [40]:
test_infer = infer_test_data(sess,infer_pred_idx_seq,test_agent.orig_data,word2idx,batch_size=1000)

In [46]:
print("Infer samples :")
for i in np.random.choice(len(test_agent.orig_data) , 10 , replace=False):
    print("  Input : " , " ".join(test_agent.orig_data[i]))
    print("  Infer : " , test_infer[i])
    print()

Input :  SOS 我 看见 快乐 在 对 我 笑 EOS r v a p v NOP iao NOE 5 NOR
Infer :  我 忍住 沉默 在 咆哮

Input :  SOS 这 说明 EOS n n uv v NOP ei NOE 4 NOR
Infer :  眼光 无力 地 凋萎

Input :  SOS 雨 不再 落下 EOS d r m n NOP ang NOE 4 NOR
Infer :  就 这样 一点 信仰

Input :  SOS 野中 EOS zg NOP ong NOE 1 NOR
Infer :  有恃无恐

Input :  SOS 凝固 发慌 的 时光 EOS c r d v n p NOP ai NOE 6 NOR
Infer :  而 我 已 没有 一遭 在

Input :  SOS 不让 我 再 为情所伤 EOS r v v p n f v NOP uo NOE 7 NOR
Infer :  我 想 站 在 梦 里 坠落

Input :  SOS 那些 路人甲 们 凭 什么 发言 惹人讨厌 准备 惊艳 EOS r c v n n uj nr NOP ie NOE 7 NOR
Infer :  他们 无为 米粮川 艾伦 线装书 的 纯氧

Input :  SOS 花若离 枝 随 莲 去 EOS v d v c NOP i NOE 4 NOR
Infer :  爱 不 舍 所以

Input :  SOS 算式 多 高深 多 艰涩 也 不想 停 EOS v v v r v d m a NOP ing NOE 8 NOR
Infer :  可 能 让 我 有 太 多 平静

Input :  SOS 叫 我 继续 追寻 EOS r uj i NOP ong NOE 3 NOR
Infer :  一样 的 海阔天空



In [53]:
def save_infer(data,name):
    path = os.path.join(exp_folder,name)
    print("Save at '{}'".format( path))
    with open( path, "w") as f:
        for s in data:
            s = "".join(s.split())
            if(len(s) == 0):
                s = np.random.choice(idx2word[4::])
            f.write(s+"\n")

def save_infer_seg(data,name):
    path = os.path.join(exp_folder,name)
    print("Save at '{}'".format( path))
    with open( path, "w") as f:
        for s in data:
            if(len(s) == 0):
                s = np.random.choice(idx2word[4::])
            f.write(s+"\n")


In [54]:
save_infer(test_infer,"infer_output.txt")
save_infer_seg(test_infer,"infer_seg.txt")

Save at './Orig_data_ver0/infer_output.txt'
Save at './Orig_data_ver0/infer_seg.txt'
